# 법정동 코드(5자리)
#### URL: https://www.code.go.kr/stdcode/regCodeL.do (.txt파일로 직접 다운로드)

In [5]:
!pip3 install chardet


  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
Using cached chardet-5.2.0-py3-none-any.whl (199 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


In [ ]:
import pymysql
from datetime import datetime
import os
from dotenv import load_dotenv
import chardet

load_dotenv()


# DB 연결
conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()


# 파일 인코딩 자동 감지

file_path = "법정동코드 전체자료.txt"

with open(file_path, "rb") as f:
    rawdata = f.read(10000)
    detected = chardet.detect(rawdata)
    encoding = detected["encoding"]
    print(f"감지된 인코딩: {encoding}")


# 법정동코드 파일 읽기

lawd_data = set()

with open(file_path, "r", encoding=encoding) as f:
    for line in f:
        parts = line.strip().split()
        if not parts:
            continue

        code = parts[0]
        if code.isdigit() and len(code) == 10:
            lawd_cd = code[:5]
            if len(parts) >= 3:
                region_nm = f"{parts[1]} {parts[2]}"
            else:
                region_nm = parts[1] if len(parts) >= 2 else "Unknown"
            lawd_data.add((lawd_cd, region_nm))


# state 테이블에 저장

lawd_data = sorted(list(lawd_data), key=lambda x: x[0])
print(f"총 {len(lawd_data)}개 시군구 코드 추출 완료")

insert_sql = """
    INSERT INTO state (region_cc, region_nm, created_at)
    VALUES (%s, %s, %s)
"""

for region_cc, region_nm in lawd_data:
    created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute(insert_sql, (region_cc, region_nm, created_at))

conn.commit()
conn.close()

print("🎯 state 테이블 저장 완료 ")


🔍 감지된 인코딩: EUC-KR
✅ 총 534개 시군구 코드 추출 완료
🎯 state 테이블 저장 완료 ✅


# 아파트 매매 실거래가
##### URL: https://www.data.go.kr/data/15126469/openapi.do#tab_layer_detail_function

In [8]:
!pip3 install xmltodict


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


In [ ]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time


# 환경 변수 로드

load_dotenv()


# DB 연결

conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()


# state 테이블에서 시군구 코드 불러오기

cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state")
regions = cursor.fetchall()
print(f"DB에서 {len(regions)}개 지역코드 불러옴")


# API 설정

service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
deal_ymd = "202509"  # 최근 공개된 월 (YYYYMM)


# 각 지역별 평균 거래가격 조회 및 state 업데이트

for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f"{region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            if amt.isdigit():
                deal_prices.append(int(amt))

        if not deal_prices:
            print(f"❗ {region_nm}: 거래금액 없음")
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        cursor.execute("""
            UPDATE state 
            SET apartment_price = %s,
                created_at = %s
            WHERE region_cc = %s
        """, (avg_price, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), region_cc))
        conn.commit()

        print(f"{region_nm} ({region_cc}) 평균 거래금액: {avg_price:,}만원 업데이트 완료")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)  # API 요청 간격 조정

conn.close()
print("state 테이블 apartment_price 업데이트 완료 ")

📍 DB에서 534개 지역코드 불러옴
❌ 서울특별시 존재 처리 중 오류: 'NoneType' object has no attribute 'get'
🏢 서울특별시 종로구 (11110) 평균 거래금액: 139,536만원 업데이트 완료
🏢 서울특별시 중구 (11140) 평균 거래금액: 140,040만원 업데이트 완료
🏢 서울특별시 용산구 (11170) 평균 거래금액: 250,350만원 업데이트 완료
🏢 서울특별시 성동구 (11200) 평균 거래금액: 181,740만원 업데이트 완료
🏢 서울특별시 광진구 (11215) 평균 거래금액: 151,770만원 업데이트 완료
🏢 서울특별시 동대문구 (11230) 평균 거래금액: 119,070만원 업데이트 완료
🏢 서울특별시 중랑구 (11260) 평균 거래금액: 89,480만원 업데이트 완료
🏢 서울특별시 성북구 (11290) 평균 거래금액: 96,440만원 업데이트 완료
🏢 서울특별시 강북구 (11305) 평균 거래금액: 72,397만원 업데이트 완료
🏢 서울특별시 도봉구 (11320) 평균 거래금액: 56,730만원 업데이트 완료
🏢 서울특별시 노원구 (11350) 평균 거래금액: 63,320만원 업데이트 완료
🏢 서울특별시 은평구 (11380) 평균 거래금액: 115,840만원 업데이트 완료
🏢 서울특별시 서대문구 (11410) 평균 거래금액: 121,415만원 업데이트 완료
🏢 서울특별시 마포구 (11440) 평균 거래금액: 228,650만원 업데이트 완료
🏢 서울특별시 양천구 (11470) 평균 거래금액: 144,520만원 업데이트 완료
🏢 서울특별시 강서구 (11500) 평균 거래금액: 90,350만원 업데이트 완료
🏢 서울특별시 구로구 (11530) 평균 거래금액: 79,930만원 업데이트 완료
🏢 서울특별시 금천구 (11545) 평균 거래금액: 72,620만원 업데이트 완료
🏢 서울특별시 영등포구 (11560) 평균 거래금액: 114,320만원 업데이트 완료
🏢 서울특별시 동작구 (11590) 평균 거래금액: 14

# 연립/다세대 주택
##### URL: https://www.data.go.kr/data/15126467/openapi.do

In [ ]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time


#  환경 변수 로드

load_dotenv()


# DB 연결

conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()


# state 테이블에서 시군구 코드 불러오기

cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state")
regions = cursor.fetchall()
print(f"DB에서 {len(regions)}개 지역코드 불러옴")


# API 설정

service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcRHTrade/getRTMSDataSvcRHTrade" 
deal_ymd = "202509"  # 최근 공개된 월 (YYYYMM)


# 각 지역별 평균 거래가격 조회 및 state 업데이트

for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f"{region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            if amt.isdigit():
                deal_prices.append(int(amt))

        if not deal_prices:
            print(f"{region_nm}: 거래금액 없음")
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        cursor.execute("""
            UPDATE state 
            SET multi_price = %s,
                created_at = %s
            WHERE region_cc = %s
        """, (avg_price, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), region_cc))
        conn.commit()

        print(f"🏢 {region_nm} ({region_cc}) 평균 거래금액: {avg_price:,}만원 → multi_price 업데이트 완료")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)  # API 요청 간격 조정

conn.close()
print("state 테이블 multi_price 컬럼 업데이트 완료 ")


📍 DB에서 282개 지역코드 불러옴
🏢 서울특별시 종로구 (11110) 평균 거래금액: 50,400만원 → multi_price 업데이트 완료
🏢 서울특별시 중구 (11140) 평균 거래금액: 47,430만원 → multi_price 업데이트 완료
🏢 서울특별시 용산구 (11170) 평균 거래금액: 81,740만원 → multi_price 업데이트 완료
🏢 서울특별시 성동구 (11200) 평균 거래금액: 91,100만원 → multi_price 업데이트 완료
🏢 서울특별시 광진구 (11215) 평균 거래금액: 31,470만원 → multi_price 업데이트 완료
🏢 서울특별시 동대문구 (11230) 평균 거래금액: 41,368만원 → multi_price 업데이트 완료
🏢 서울특별시 중랑구 (11260) 평균 거래금액: 33,695만원 → multi_price 업데이트 완료
🏢 서울특별시 성북구 (11290) 평균 거래금액: 31,500만원 → multi_price 업데이트 완료
🏢 서울특별시 강북구 (11305) 평균 거래금액: 24,170만원 → multi_price 업데이트 완료
🏢 서울특별시 도봉구 (11320) 평균 거래금액: 25,455만원 → multi_price 업데이트 완료
🏢 서울특별시 노원구 (11350) 평균 거래금액: 24,505만원 → multi_price 업데이트 완료
🏢 서울특별시 은평구 (11380) 평균 거래금액: 34,056만원 → multi_price 업데이트 완료
🏢 서울특별시 서대문구 (11410) 평균 거래금액: 48,590만원 → multi_price 업데이트 완료
🏢 서울특별시 마포구 (11440) 평균 거래금액: 44,960만원 → multi_price 업데이트 완료
🏢 서울특별시 양천구 (11470) 평균 거래금액: 26,580만원 → multi_price 업데이트 완료
🏢 서울특별시 강서구 (11500) 평균 거래금액: 23,510만원 → multi_price 업데이트 완료
🏢 서울특별시 구로구 (11530

# 오피스텔
##### URL: https://www.data.go.kr/data/15126464/openapi.do

In [ ]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time


# 환경 변수 로드

load_dotenv()


# DB 연결

conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()


# state 테이블에서 시군구 코드 불러오기

cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state")
regions = cursor.fetchall()
print(f"DB에서 {len(regions)}개 지역코드 불러옴")


# officetel_price 컬럼 자동 생성 (없을 경우)

cursor.execute("""
    SELECT COUNT(*) 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_SCHEMA = %s 
      AND TABLE_NAME = 'state' 
      AND COLUMN_NAME = 'officetel_price';
""", (os.getenv("database"),))
exists = cursor.fetchone()[0]

if not exists:
    print("officetel_price 컬럼이 없어 새로 생성합니다...")
    cursor.execute("ALTER TABLE state ADD COLUMN officetel_price BIGINT DEFAULT NULL;")
    conn.commit()
else:
    print("officetel_price 컬럼이 이미 존재합니다.")


# API 설정 (오피스텔 매매 실거래가)

service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcOffiTrade/getRTMSDataSvcOffiTrade"
deal_ymd = "202509"  # 최근 공개된 월 (YYYYMM)


# 지역별 평균 거래가격 조회 및 state 업데이트

for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f" {region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            if amt.isdigit():
                deal_prices.append(int(amt) * 10000)  

        if not deal_prices:
            print(f"{region_nm}: 거래금액 없음")
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        cursor.execute("""
            UPDATE state 
            SET officetel_price = %s,
                created_at = %s
            WHERE region_cc = %s
        """, (avg_price, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), region_cc))
        conn.commit()

        print(f"{region_nm} ({region_cc}) 평균 거래금액: {avg_price:,}원 → officetel_price 업데이트 완료")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)

conn.close()
print("state 테이블 officetel_price 컬럼 업데이트 완료 ")


📍 DB에서 282개 지역코드 불러옴
⚙️ officetel_price 컬럼이 없어 새로 생성합니다...
🏢 서울특별시 종로구 (11110) 평균 거래금액: 565,500,000원 → officetel_price 업데이트 완료
🏢 서울특별시 중구 (11140) 평균 거래금액: 351,450,000원 → officetel_price 업데이트 완료
🏢 서울특별시 용산구 (11170) 평균 거래금액: 489,700,000원 → officetel_price 업데이트 완료
🏢 서울특별시 성동구 (11200) 평균 거래금액: 173,850,000원 → officetel_price 업데이트 완료
🏢 서울특별시 광진구 (11215) 평균 거래금액: 361,825,000원 → officetel_price 업데이트 완료
🏢 서울특별시 동대문구 (11230) 평균 거래금액: 184,300,000원 → officetel_price 업데이트 완료
🏢 서울특별시 중랑구 (11260) 평균 거래금액: 349,285,714원 → officetel_price 업데이트 완료
🏢 서울특별시 성북구 (11290) 평균 거래금액: 151,666,667원 → officetel_price 업데이트 완료
🏢 서울특별시 강북구 (11305) 평균 거래금액: 129,600,000원 → officetel_price 업데이트 완료
🏢 서울특별시 도봉구 (11320) 평균 거래금액: 191,300,000원 → officetel_price 업데이트 완료
🏢 서울특별시 노원구 (11350) 평균 거래금액: 180,375,000원 → officetel_price 업데이트 완료
🏢 서울특별시 은평구 (11380) 평균 거래금액: 197,500,000원 → officetel_price 업데이트 완료
🏢 서울특별시 서대문구 (11410) 평균 거래금액: 262,800,000원 → officetel_price 업데이트 완료
🏢 서울특별시 마포구 (11440) 평균 거래금액: 429,850,000원 → officetel_pr

# 단독/다가구 주택
##### URL: https://www.data.go.kr/data/15126465/openapi.do

In [ ]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time

load_dotenv()

conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()

# state 테이블 지역코드 불러오기
cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state")
regions = cursor.fetchall()
print(f"DB에서 {len(regions)}개 지역코드 불러옴")

# detached_price 컬럼 자동 생성 (없으면 추가)
cursor.execute("""
    SELECT COUNT(*) 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_SCHEMA = %s 
      AND TABLE_NAME = 'state' 
      AND COLUMN_NAME = 'detached_price';
""", (os.getenv("database"),))
exists = cursor.fetchone()[0]

if not exists:
    print(" detached_price 컬럼이 없어 새로 생성합니다...")
    cursor.execute("ALTER TABLE state ADD COLUMN detached_price BIGINT DEFAULT NULL;")
    conn.commit()
else:
    print("detached_price 컬럼이 이미 존재합니다.")

# 단독/다가구 매매 실거래가 API 설정
service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcSHTrade/getRTMSDataSvcSHTrade"
deal_ymd = "202509"

# 지역별 평균 거래가격 조회 및 state 업데이트
for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f"{region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            if amt.isdigit():
                deal_prices.append(int(amt) * 10000)  # 만원 → 원 단위 변환

        if not deal_prices:
            print(f"{region_nm}: 거래금액 없음")
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        cursor.execute("""
            UPDATE state 
            SET detached_price = %s,
                created_at = %s
            WHERE region_cc = %s
        """, (avg_price, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), region_cc))
        conn.commit()

        print(f"{region_nm} ({region_cc}) 평균 거래금액: {avg_price:,}원 → detached_price 업데이트 완료")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)

conn.close()
print("state 테이블 detached_price 컬럼 업데이트 완료 ")


📍 DB에서 282개 지역코드 불러옴
⚙️ detached_price 컬럼이 없어 새로 생성합니다...
🏠 서울특별시 종로구 (11110) 평균 거래금액: 1,828,700,000원 → detached_price 업데이트 완료
🏠 서울특별시 중구 (11140) 평균 거래금액: 1,952,119,000원 → detached_price 업데이트 완료
🏠 서울특별시 용산구 (11170) 평균 거래금액: 1,508,500,000원 → detached_price 업데이트 완료
🏠 서울특별시 성동구 (11200) 평균 거래금액: 2,006,500,000원 → detached_price 업데이트 완료
🏠 서울특별시 광진구 (11215) 평균 거래금액: 1,999,000,000원 → detached_price 업데이트 완료
🏠 서울특별시 동대문구 (11230) 평균 거래금액: 1,394,462,000원 → detached_price 업데이트 완료
🏠 서울특별시 중랑구 (11260) 평균 거래금액: 873,700,000원 → detached_price 업데이트 완료
🏠 서울특별시 성북구 (11290) 평균 거래금액: 1,190,300,000원 → detached_price 업데이트 완료
🏠 서울특별시 강북구 (11305) 평균 거래금액: 747,600,000원 → detached_price 업데이트 완료
🏠 서울특별시 도봉구 (11320) 평균 거래금액: 614,285,714원 → detached_price 업데이트 완료
🏠 서울특별시 노원구 (11350) 평균 거래금액: 700,000,000원 → detached_price 업데이트 완료
🏠 서울특별시 은평구 (11380) 평균 거래금액: 1,517,612,500원 → detached_price 업데이트 완료
🏠 서울특별시 서대문구 (11410) 평균 거래금액: 1,265,374,000원 → detached_price 업데이트 완료
🏠 서울특별시 마포구 (11440) 평균 거래금액: 1,463,000,000원 → detach

# 경기도 추가

In [2]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# ✅ 환경 변수 로드
load_dotenv()

# ✅ DB 연결
conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()

# ✅ state 테이블에서 경기도 지역코드만 불러오기
cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state WHERE region_cc LIKE '41%'")
regions = cursor.fetchall()
print(f"📍 DB에서 {len(regions)}개 경기도 지역코드 불러옴")

# ✅ API 설정
service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
deal_ymd = "202509"  # 최근 공개된 월 (YYYYMM)

# ✅ 모든 경기도 지역 평균 누적 리스트
all_prices = []

# ✅ 각 시군구별 평균 구하기
for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f"{region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            if amt.isdigit():
                deal_prices.append(int(amt))

        if not deal_prices:
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        all_prices.append(avg_price)
        print(f"✅ {region_nm} ({region_cc}) 평균: {avg_price:,}만원")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)  # API 요청 간격

# ✅ 경기도 전체 평균 계산 및 state 반영
if all_prices:
    gyeonggi_avg = round(sum(all_prices) / len(all_prices))
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    print(f"\n🎯 경기도 전체 평균 아파트 거래가격: {gyeonggi_avg:,}만원")

    # ✅ 기존에 41000 코드가 있는지 확인
    cursor.execute("SELECT COUNT(*) FROM state WHERE region_cc = '41000'")
    exists = cursor.fetchone()[0]

    if exists:
        # UPDATE
        cursor.execute("""
            UPDATE state
            SET apartment_price = %s, created_at = %s
            WHERE region_cc = '41000'
        """, (gyeonggi_avg, now))
        print("✅ 기존 41000 레코드 업데이트 완료")
    else:
        # INSERT
        cursor.execute("""
            INSERT INTO state (region_cc, region_nm, apartment_price, created_at)
            VALUES (%s, %s, %s, %s)
        """, ('41000', '경기도', gyeonggi_avg, now))
        print("✅ 새 경기도 레코드 생성 완료")

    conn.commit()
else:
    print("❌ 경기도 데이터가 없습니다.")

conn.close()
print("작업 완료 ✅")


📍 DB에서 44개 경기도 지역코드 불러옴
✅ 경기도 수원시 (41111) 평균: 61,810만원
✅ 경기도 수원시 (41113) 평균: 42,790만원
✅ 경기도 수원시 (41115) 평균: 58,780만원
✅ 경기도 수원시 (41117) 평균: 95,555만원
✅ 경기도 성남시 (41131) 평균: 131,200만원
✅ 경기도 성남시 (41133) 평균: 78,630만원
✅ 경기도 성남시 (41135) 평균: 165,480만원
✅ 경기도 의정부시 (41150) 평균: 43,310만원
✅ 경기도 안양시 (41171) 평균: 59,495만원
✅ 경기도 안양시 (41173) 평균: 80,900만원
✅ 경기도 부천시 (41192) 평균: 53,110만원
✅ 경기도 부천시 (41194) 평균: 47,140만원
✅ 경기도 부천시 (41196) 평균: 34,170만원
✅ 경기도 광명시 (41210) 평균: 85,130만원
✅ 경기도 평택시 (41220) 평균: 26,596만원
✅ 경기도 동두천시 (41250) 평균: 19,345만원
✅ 경기도 안산시 (41271) 평균: 41,410만원
✅ 경기도 안산시 (41273) 평균: 50,297만원
✅ 경기도 고양시 (41281) 평균: 46,020만원
✅ 경기도 고양시 (41285) 평균: 71,280만원
✅ 경기도 고양시 (41287) 평균: 49,840만원
✅ 경기도 과천시 (41290) 평균: 207,330만원
✅ 경기도 구리시 (41310) 평균: 72,670만원
✅ 경기도 남양주시 (41360) 평균: 68,444만원
✅ 경기도 오산시 (41370) 평균: 35,680만원
✅ 경기도 시흥시 (41390) 평균: 33,814만원
✅ 경기도 군포시 (41410) 평균: 50,430만원
✅ 경기도 의왕시 (41430) 평균: 91,300만원
✅ 경기도 하남시 (41450) 평균: 111,770만원
✅ 경기도 용인시 (41461) 평균: 31,640만원
✅ 경기도 용인시 (41463) 평균: 52,630만원
✅ 경기도 용인

# 경기도 연립다세대 추가

In [3]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# ✅ 환경 변수 로드
load_dotenv()

# ✅ DB 연결
conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()

# ✅ 경기도 지역코드만 불러오기
cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state WHERE region_cc LIKE '41%'")
regions = cursor.fetchall()
print(f"📍 DB에서 {len(regions)}개 경기도 지역코드 불러옴")

# ✅ API 설정
service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcRHTrade/getRTMSDataSvcRHTrade"  # 연립/다세대 실거래가
deal_ymd = "202509"  # 최근 공개된 월 (YYYYMM)

# ✅ 경기도 전체 평균 누적 리스트
all_prices = []

# ✅ 각 시군구별 평균 계산
for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f"{region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            if amt.isdigit():
                deal_prices.append(int(amt))

        if not deal_prices:
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        all_prices.append(avg_price)
        print(f"🏢 {region_nm} ({region_cc}) 평균: {avg_price:,}만원")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)  # API 요청 간격

# ✅ 경기도 전체 평균 계산 후 state 테이블 업데이트
if all_prices:
    gyeonggi_multi_avg = round(sum(all_prices) / len(all_prices))
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    print(f"\n🎯 경기도 전체 연립·다세대 평균 거래금액: {gyeonggi_multi_avg:,}만원")

    # ✅ 기존에 region_cc = 41000 존재 여부 확인
    cursor.execute("SELECT COUNT(*) FROM state WHERE region_cc = '41000'")
    exists = cursor.fetchone()[0]

    if exists:
        # UPDATE
        cursor.execute("""
            UPDATE state
            SET multi_price = %s, created_at = %s
            WHERE region_cc = '41000'
        """, (gyeonggi_multi_avg, now))
        print("✅ 기존 41000 레코드 multi_price 업데이트 완료")
    else:
        # INSERT
        cursor.execute("""
            INSERT INTO state (region_cc, region_nm, multi_price, created_at)
            VALUES (%s, %s, %s, %s)
        """, ('41000', '경기도', gyeonggi_multi_avg, now))
        print("✅ 새 경기도 레코드 생성 완료")

    conn.commit()
else:
    print("❌ 경기도 연립·다세대 데이터가 없습니다.")

conn.close()
print("작업 완료 ✅")


📍 DB에서 45개 경기도 지역코드 불러옴
🏢 경기도 수원시 (41111) 평균: 14,364만원
🏢 경기도 수원시 (41113) 평균: 20,800만원
🏢 경기도 수원시 (41115) 평균: 20,880만원
🏢 경기도 수원시 (41117) 평균: 93,775만원
🏢 경기도 성남시 (41131) 평균: 36,480만원
🏢 경기도 성남시 (41133) 평균: 27,630만원
🏢 경기도 성남시 (41135) 평균: 106,150만원
🏢 경기도 의정부시 (41150) 평균: 16,640만원
🏢 경기도 안양시 (41171) 평균: 24,923만원
🏢 경기도 안양시 (41173) 평균: 37,985만원
🏢 경기도 부천시 (41192) 평균: 17,870만원
🏢 경기도 부천시 (41194) 평균: 19,440만원
🏢 경기도 부천시 (41196) 평균: 14,340만원
🏢 경기도 광명시 (41210) 평균: 41,860만원
🏢 경기도 평택시 (41220) 평균: 10,590만원
🏢 경기도 동두천시 (41250) 평균: 4,840만원
🏢 경기도 안산시 (41271) 평균: 10,237만원
🏢 경기도 안산시 (41273) 평균: 21,940만원
🏢 경기도 고양시 (41281) 평균: 33,585만원
🏢 경기도 고양시 (41285) 평균: 29,605만원
🏢 경기도 고양시 (41287) 평균: 22,110만원
🏢 경기도 과천시 (41290) 평균: 84,800만원
🏢 경기도 구리시 (41310) 평균: 37,965만원
🏢 경기도 남양주시 (41360) 평균: 22,230만원
🏢 경기도 오산시 (41370) 평균: 12,940만원
🏢 경기도 시흥시 (41390) 평균: 10,100만원
🏢 경기도 군포시 (41410) 평균: 30,420만원
🏢 경기도 의왕시 (41430) 평균: 32,180만원
🏢 경기도 하남시 (41450) 평균: 43,830만원
🏢 경기도 용인시 (41461) 평균: 13,345만원
🏢 경기도 용인시 (41463) 평균: 56,065만원
🏢 경기도 용인시 (4

# 경기도 오피스텔 평균 가격

In [4]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# ✅ 환경 변수 로드
load_dotenv()

# ✅ DB 연결
conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()

# ✅ 경기도 지역코드만 불러오기
cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state WHERE region_cc LIKE '41%'")
regions = cursor.fetchall()
print(f"📍 DB에서 {len(regions)}개 경기도 지역코드 불러옴")

# ✅ officetel_price 컬럼 자동 생성 (없으면 추가)
cursor.execute("""
    SELECT COUNT(*) 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_SCHEMA = %s 
      AND TABLE_NAME = 'state' 
      AND COLUMN_NAME = 'officetel_price';
""", (os.getenv("database"),))
exists = cursor.fetchone()[0]

if not exists:
    print("officetel_price 컬럼이 없어 새로 생성합니다...")
    cursor.execute("ALTER TABLE state ADD COLUMN officetel_price BIGINT DEFAULT NULL;")
    conn.commit()
else:
    print("officetel_price 컬럼이 이미 존재합니다.")

# ✅ API 설정 (오피스텔 매매 실거래가)
service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcOffiTrade/getRTMSDataSvcOffiTrade"
deal_ymd = "202509"  # 최근 공개된 월 (YYYYMM)

# ✅ 경기도 평균 누적 리스트
all_prices = []

# ✅ 각 시군구별 평균 계산
for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f"{region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            # 만원 단위를 원 단위로 변환
            if amt.isdigit():
                deal_prices.append(int(amt) * 10000)

        if not deal_prices:
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        all_prices.append(avg_price)
        print(f"🏢 {region_nm} ({region_cc}) 평균: {avg_price:,}원")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)

# ✅ 경기도 전체 평균 계산 후 state 반영
if all_prices:
    gyeonggi_office_avg = round(sum(all_prices) / len(all_prices))
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    print(f"\n🎯 경기도 전체 오피스텔 평균 거래가격: {gyeonggi_office_avg:,}원")

    # ✅ 41000 행 존재 여부 확인
    cursor.execute("SELECT COUNT(*) FROM state WHERE region_cc = '41000'")
    exists = cursor.fetchone()[0]

    if exists:
        # UPDATE
        cursor.execute("""
            UPDATE state
            SET officetel_price = %s, created_at = %s
            WHERE region_cc = '41000'
        """, (gyeonggi_office_avg, now))
        print("✅ 기존 41000 레코드 officetel_price 업데이트 완료")
    else:
        # INSERT
        cursor.execute("""
            INSERT INTO state (region_cc, region_nm, officetel_price, created_at)
            VALUES (%s, %s, %s, %s)
        """, ('41000', '경기도', gyeonggi_office_avg, now))
        print("✅ 새 경기도 레코드 생성 완료")

    conn.commit()
else:
    print("❌ 경기도 오피스텔 데이터가 없습니다.")

conn.close()
print("작업 완료 ✅")


📍 DB에서 45개 경기도 지역코드 불러옴
officetel_price 컬럼이 이미 존재합니다.
🏢 경기도 수원시 (41111) 평균: 255,000,000원
🏢 경기도 수원시 (41113) 평균: 147,444,444원
🏢 경기도 수원시 (41115) 평균: 179,100,000원
🏢 경기도 수원시 (41117) 평균: 339,000,000원
🏢 경기도 성남시 (41131) 평균: 490,300,000원
🏢 경기도 성남시 (41133) 평균: 212,100,000원
🏢 경기도 성남시 (41135) 평균: 598,380,000원
🏢 경기도 의정부시 (41150) 평균: 191,260,000원
🏢 경기도 안양시 (41171) 평균: 193,000,000원
🏢 경기도 안양시 (41173) 평균: 274,100,000원
🏢 경기도 부천시 (41192) 평균: 192,450,000원
🏢 경기도 부천시 (41194) 평균: 218,400,000원
🏢 경기도 부천시 (41196) 평균: 190,000,000원
🏢 경기도 광명시 (41210) 평균: 307,821,000원
🏢 경기도 평택시 (41220) 평균: 178,666,667원
경기도 동두천시 처리 중 오류: 'NoneType' object has no attribute 'get'
🏢 경기도 안산시 (41271) 평균: 205,100,000원
🏢 경기도 안산시 (41273) 평균: 115,750,000원
🏢 경기도 고양시 (41281) 평균: 420,119,000원
🏢 경기도 고양시 (41285) 평균: 265,100,000원
🏢 경기도 고양시 (41287) 평균: 289,800,000원
🏢 경기도 과천시 (41290) 평균: 719,666,667원
🏢 경기도 구리시 (41310) 평균: 146,360,000원
🏢 경기도 남양주시 (41360) 평균: 112,900,000원
🏢 경기도 오산시 (41370) 평균: 135,714,286원
🏢 경기도 시흥시 (41390) 평균: 135,350,000원
🏢 경기도 군포시 

# 경기도 단독/다가구 주택 평균 가격

In [5]:
import pymysql
import requests
import xmltodict
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# ✅ 환경 변수 로드
load_dotenv()

# ✅ DB 연결
conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()

# ✅ 경기도 지역코드만 불러오기
cursor.execute("SELECT DISTINCT region_cc, region_nm FROM state WHERE region_cc LIKE '41%'")
regions = cursor.fetchall()
print(f"📍 DB에서 {len(regions)}개 경기도 지역코드 불러옴")

# ✅ detached_price 컬럼 자동 생성 (없으면 추가)
cursor.execute("""
    SELECT COUNT(*) 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_SCHEMA = %s 
      AND TABLE_NAME = 'state' 
      AND COLUMN_NAME = 'detached_price';
""", (os.getenv("database"),))
exists = cursor.fetchone()[0]

if not exists:
    print("detached_price 컬럼이 없어 새로 생성합니다...")
    cursor.execute("ALTER TABLE state ADD COLUMN detached_price BIGINT DEFAULT NULL;")
    conn.commit()
else:
    print("detached_price 컬럼이 이미 존재합니다.")

# ✅ 단독/다가구 매매 실거래가 API 설정
service_key = "NuxFc71Tw1ot6GcGZW6p+7/DV3j+u6RstpL3kqApCXcM/hvuDuZ+q6Q4TxxTrESLw/aCXSrV+ykPkkOI2reJoQ=="
base_url = "https://apis.data.go.kr/1613000/RTMSDataSvcSHTrade/getRTMSDataSvcSHTrade"
deal_ymd = "202509"  # 최근 공개된 월 (YYYYMM)

# ✅ 경기도 전체 평균 누적 리스트
all_prices = []

# ✅ 각 시군구별 평균 계산
for region_cc, region_nm in regions:
    params = {
        "serviceKey": service_key,
        "LAWD_CD": region_cc,
        "DEAL_YMD": deal_ymd
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"요청 실패 ({region_nm}): {response.status_code}")
            continue

        data = xmltodict.parse(response.text)
        items = (
            data.get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if not items:
            print(f"{region_nm} ({region_cc}): 데이터 없음")
            continue

        if isinstance(items, dict):
            items = [items]

        deal_prices = []
        for item in items:
            amt = item.get("dealAmount", "").replace(",", "").strip()
            if amt.isdigit():
                deal_prices.append(int(amt) * 10000)  # 만원 → 원 단위

        if not deal_prices:
            continue

        avg_price = round(sum(deal_prices) / len(deal_prices))
        all_prices.append(avg_price)
        print(f"🏠 {region_nm} ({region_cc}) 평균: {avg_price:,}원")

    except Exception as e:
        print(f"{region_nm} 처리 중 오류: {e}")

    time.sleep(0.3)

# ✅ 경기도 전체 평균 계산 및 DB 반영
if all_prices:
    gyeonggi_detached_avg = round(sum(all_prices) / len(all_prices))
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    print(f"\n🎯 경기도 전체 단독·다가구 평균 거래금액: {gyeonggi_detached_avg:,}원")

    # ✅ 41000 코드 존재 여부 확인
    cursor.execute("SELECT COUNT(*) FROM state WHERE region_cc = '41000'")
    exists = cursor.fetchone()[0]

    if exists:
        # UPDATE
        cursor.execute("""
            UPDATE state
            SET detached_price = %s, created_at = %s
            WHERE region_cc = '41000'
        """, (gyeonggi_detached_avg, now))
        print("✅ 기존 41000 레코드 detached_price 업데이트 완료")
    else:
        # INSERT
        cursor.execute("""
            INSERT INTO state (region_cc, region_nm, detached_price, created_at)
            VALUES (%s, %s, %s, %s)
        """, ('41000', '경기도', gyeonggi_detached_avg, now))
        print("✅ 새 경기도 레코드 생성 완료")

    conn.commit()
else:
    print("❌ 경기도 단독·다가구 데이터가 없습니다.")

conn.close()
print("작업 완료 ✅")


📍 DB에서 45개 경기도 지역코드 불러옴
detached_price 컬럼이 이미 존재합니다.
🏠 경기도 수원시 (41111) 평균: 452,000,000원
🏠 경기도 수원시 (41113) 평균: 397,250,000원
🏠 경기도 수원시 (41115) 평균: 528,430,000원
🏠 경기도 수원시 (41117) 평균: 823,571,429원
🏠 경기도 성남시 (41131) 평균: 705,940,000원
🏠 경기도 성남시 (41133) 평균: 618,250,000원
🏠 경기도 성남시 (41135) 평균: 1,596,250,000원
🏠 경기도 의정부시 (41150) 평균: 548,480,000원
🏠 경기도 안양시 (41171) 평균: 810,000,000원
🏠 경기도 안양시 (41173) 평균: 1,106,666,667원
🏠 경기도 부천시 (41192) 평균: 439,900,000원
🏠 경기도 부천시 (41194) 평균: 307,000,000원
🏠 경기도 부천시 (41196) 평균: 1,210,000,000원
🏠 경기도 광명시 (41210) 평균: 805,000,000원
🏠 경기도 평택시 (41220) 평균: 579,750,000원
🏠 경기도 동두천시 (41250) 평균: 122,357,143원
🏠 경기도 안산시 (41271) 평균: 922,220,000원
🏠 경기도 안산시 (41273) 평균: 538,641,000원
🏠 경기도 고양시 (41281) 평균: 539,388,889원
🏠 경기도 고양시 (41285) 평균: 897,122,000원
🏠 경기도 고양시 (41287) 평균: 80,000,000원
🏠 경기도 과천시 (41290) 평균: 1,702,000,000원
🏠 경기도 구리시 (41310) 평균: 690,000,000원
🏠 경기도 남양주시 (41360) 평균: 579,375,000원
🏠 경기도 오산시 (41370) 평균: 603,625,000원
🏠 경기도 시흥시 (41390) 평균: 591,666,667원
경기도 군포시 처리 중 오류: 'NoneType'

# 서울특별시를 제외한 나머지 지역 평균으로 묶기

In [7]:
import pymysql
from datetime import datetime
import os
from dotenv import load_dotenv

# ✅ 환경 변수 로드
load_dotenv()

# ✅ DB 연결
conn = pymysql.connect(
    host=os.getenv("host"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    database=os.getenv("database"),
    charset="utf8mb4"
)
cursor = conn.cursor()

print("📊 시도별 평균가격 계산 중 (서울특별시 제외)...")

# ✅ 1️⃣ 시도 단위별 평균 계산 (서울 제외)
query = """
SELECT 
    CONCAT(LEFT(region_cc, 2), '000') AS mapped_region,
    ROUND(AVG(apartment_price)) AS avg_apartment_price,
    ROUND(AVG(multi_price)) AS avg_multi_price,
    ROUND(AVG(officetel_price)) AS avg_officetel_price,
    ROUND(AVG(detached_price)) AS avg_detached_price
FROM state
WHERE LEFT(region_cc, 2) <> '11'
GROUP BY CONCAT(LEFT(region_cc, 2), '000')
ORDER BY mapped_region;
"""
cursor.execute(query)
rows = cursor.fetchall()

print(f"✅ 총 {len(rows)}개 시·도 평균 계산 완료")

# ✅ 2️⃣ 시도 코드 → 시도명 매핑 딕셔너리
sido_map = {
    '26': '부산광역시',
    '27': '대구광역시',
    '28': '인천광역시',
    '29': '광주광역시',
    '30': '대전광역시',
    '31': '울산광역시',
    '36': '세종특별자치시',
    '41': '경기도',
    '42': '강원특별자치도',
    '43': '충청북도',
    '44': '충청남도',
    '45': '전라북도',
    '46': '전라남도',
    '47': '경상북도',
    '48': '경상남도',
    '50': '제주특별자치도',
    '51': '서울특별시',
    '52': '세종특별자치시'
}

# ✅ 3️⃣ 평균값을 대표 시도코드(26000 등)에 업데이트/생성
for mapped_region, apt_avg, multi_avg, officetel_avg, detached_avg in rows:
    sido_prefix = mapped_region[:2]
    region_nm = sido_map.get(sido_prefix, f"{sido_prefix}000 지역")
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    cursor.execute("SELECT COUNT(*) FROM state WHERE region_cc = %s", (mapped_region,))
    exists = cursor.fetchone()[0]

    if exists:
        cursor.execute("""
            UPDATE state
            SET apartment_price = %s,
                multi_price = %s,
                officetel_price = %s,
                detached_price = %s,
                region_nm = %s,
                created_at = %s
            WHERE region_cc = %s
        """, (apt_avg, multi_avg, officetel_avg, detached_avg, region_nm, now, mapped_region))
        print(f"🔄 {region_nm} ({mapped_region}) 평균 갱신 완료")
    else:
        cursor.execute("""
            INSERT INTO state (region_cc, region_nm, apartment_price, multi_price, officetel_price, detached_price, created_at)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (mapped_region, region_nm, apt_avg, multi_avg, officetel_avg, detached_avg, now))
        print(f"🆕 {region_nm} ({mapped_region}) 평균 신규 추가 완료")

conn.commit()
conn.close()

print("\n🎯 모든 시도별 평균(아파트/연립다세대/오피스텔/단독다가구) 업데이트 완료 ✅")


📊 시도별 평균가격 계산 중 (서울특별시 제외)...
✅ 총 16개 시·도 평균 계산 완료
🔄 부산광역시 (26000) 평균 갱신 완료
🔄 대구광역시 (27000) 평균 갱신 완료
🔄 인천광역시 (28000) 평균 갱신 완료
🔄 광주광역시 (29000) 평균 갱신 완료
🔄 대전광역시 (30000) 평균 갱신 완료
🔄 울산광역시 (31000) 평균 갱신 완료
🔄 세종특별자치시 (36000) 평균 갱신 완료
🔄 경기도 (41000) 평균 갱신 완료
🔄 충청북도 (43000) 평균 갱신 완료
🔄 충청남도 (44000) 평균 갱신 완료
🔄 전라남도 (46000) 평균 갱신 완료
🔄 경상북도 (47000) 평균 갱신 완료
🔄 경상남도 (48000) 평균 갱신 완료
🔄 제주특별자치도 (50000) 평균 갱신 완료
🔄 서울특별시 (51000) 평균 갱신 완료
🔄 세종특별자치시 (52000) 평균 갱신 완료

🎯 모든 시도별 평균(아파트/연립다세대/오피스텔/단독다가구) 업데이트 완료 ✅
